In [74]:
# automatically reload the libraries from time_profiling.py
import os
import sys
module_path = os.path.abspath(os.path.join('/Users/Johannes/Uni/Master/Master Arbeit/repos/matrix-operations'))
if module_path not in sys.path:
    sys.path.append(module_path)
from split import Split
%reload_ext autoreload
%autoreload 2

In [75]:
from collections import namedtuple

MatrixDimensions = namedtuple('MatrixDimensions', ['height', 'width'])

In [103]:
BUCKET = 'jmue-multiplication-benchmarks'
name_matrixA = "sq_8kx8k_bs{}k".format(bl)
name_matrixB = "sq_8kx8k_bs{}k-2"
block_size = 2000
matrix_dimensions = MatrixDimensions(8000,8000)
split_size = block_size * 2

In [109]:
event = {
    'state-machine-name': "state_machine_name",
            'executionName': "executionName",
            'matA': {
                'bucket': BUCKET,
                'folder': name_matrixA,
                'block-size': block_size,
                'rows': matrix_dimensions.height,
                'columns': matrix_dimensions.width
            },
            'matB': {
                'bucket': BUCKET,
                'folder': name_matrixB,
                'block-size': block_size,
                'rows': matrix_dimensions.height,
                'columns': matrix_dimensions.height
            },
            'result': {
                'bucket': BUCKET,
                'folder': name_matrixA + '-result',
                'block-size': block_size,
                "split": {
                    'x': 0,
                    'y': 0
                }
            },
            'split-size': block_size * 2,
            'split': 1,
            'unit': 1
        }

In [105]:
result_split = Split(event['result'], event['result']['split'], event['split-size'])

In [106]:
print result_split

{"y": 0, "x": 0, "block_size": 2000, "matrix": "jmue-multiplication-benchmarks/sq_8kx8k_bs4k-result", "split_size": 4000}


In [107]:
left_split = Split.left_inputsplit_for(event['matA'], result_split, event['unit'])
right_split = Split.right_inputsplit_for(event['matB'], result_split, event['unit'])
print left_split
print right_split

{"y": 0, "x": 0, "block_size": 2000, "matrix": "jmue-multiplication-benchmarks/sq_8kx8k_bs4k", "split_size": 4000}
{"y": 0, "x": 0, "block_size": 2000, "matrix": "jmue-multiplication-benchmarks/sq_8kx8k_bs4k-2", "split_size": 4000}


In [108]:
loaded = left_split.block(1,1)

m_1_1.npy
4000
(4000, 4000)


## Does blocksize affect download time?

In [33]:
import numpy as np
import boto3

In [34]:
s3_client = boto3.client('s3')

In [ ]:
# 4 * 122.1MB
'jmue-multiplication-benchmarks/sq_8kx8k_bs4k', range(0,2)

# 16 * 7,6MB
'jmue-multiplication-benchmarks/sq_8kx8k_bs1k', range(0,8)

In [36]:
bucket = 'jmue-multiplication-benchmarks'

In [42]:
def download_all_parts(indeces, matrix_name):
    for i in indeces:
        for j in indeces:
            with open('/tmp/tmp_filename', 'wb') as data:
                key = "{}/m_{}_{}.npy".format(matrix_name, i, j)
                s3_client.download_fileobj(bucket, key, data)

In [47]:
%%time
download_all_parts(range(0,8), 'sq_8kx8k_bs1k')

CPU times: user 3.61 s, sys: 3.6 s, total: 7.21 s
Wall time: 55.1 s


CPU times: user 3.17 s, sys: 3.22 s, total: 6.39 s
Wall time: 54.2 s

CPU times: user 3.61 s, sys: 3.6 s, total: 7.21 s
Wall time: 55.1 s

In [48]:
%%time
download_all_parts(range(0,2), 'sq_8kx8k_bs4k')

CPU times: user 3.77 s, sys: 4.58 s, total: 8.35 s
Wall time: 29.4 s


CPU times: user 3.62 s, sys: 4.54 s, total: 8.16 s
Wall time: 28.9 s

CPU times: user 3.77 s, sys: 4.58 s, total: 8.35 s
Wall time: 29.4 s